In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, datasets
from matplotlib import pyplot as plt
import collections
from models.VisionTransformer import VisionTransformerClassifier
from models.stegastamp_wm import StegaStampDecoder, StegaStampEncoder
from score import f1
import torch.optim as optim
from dataset import get_image_dataloader
import os

In [2]:
image_size = 256
num_bits = 64
if torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cuda') if torch.cuda.is_available() else 'cpu'   
    
transform = transforms.Compose([
                                transforms.Resize(image_size),
                                transforms.CenterCrop(image_size),
                                transforms.ToTensor(),
                            ]) 

In [3]:
encoder = StegaStampEncoder(image_size, 3, num_bits).to(device)
encoder.load_state_dict(torch.load('models/wm_stegastamp_encoder.pth', map_location=device, weights_only=True))
decoder = StegaStampDecoder(image_size, 3, num_bits).to(device)
decoder.load_state_dict(torch.load('models/wm_stegastamp_decoder.pth', map_location=device, weights_only=True))

<All keys matched successfully>

In [4]:
num_epochs = 100
batch_size = 64
train_loader = get_image_dataloader("./../data/images/train", transform, batch_size=batch_size, shuffle=True)
val_loader = get_image_dataloader("./../data/images/val", transform, batch_size=batch_size, shuffle=True)

model = VisionTransformerClassifier(input_resolution=image_size, layers=4, heads=8, output_dim=2).to(device)
crit = nn.CrossEntropyLoss()
opt = optim.Adam(model.parameters(), lr=1e-4)


In [ ]:
for epoch in range(num_epochs):
    tp = 1e-10
    fp = 1e-10
    fn = 1e-10
    tn = 1e-10
    for images in train_loader:
        images = images.to(device)
        watermarks = torch.randint(0, 2, (images.shape[0], num_bits)).float().to(device)
        encoded_split = images.shape[0] // 2
        watermarks[:encoded_split] = 2
        images[encoded_split:] = encoder(watermarks[encoded_split:], images[encoded_split:])
        true_labels = (watermarks[:, 0] != 2).int()
        
        
        opt.zero_grad()
        pred_scores = model(images)
        pred_labels = torch.argmax(pred_scores, dim=-1)
        loss = crit(pred_scores, true_labels)
        loss.backward()
        opt.step()
        
        tp += torch.sum((pred_labels == 1) & (pred_labels == true_labels)).item()
        tn += torch.sum((pred_labels == 0) & (pred_labels == true_labels)).item()
        fp += torch.sum((pred_labels == 1) & (pred_labels != true_labels)).item()
        fn += torch.sum((pred_labels == 0) & (pred_labels != true_labels)).item()
        print(f1(tp, tn, fp, tn))
        print(loss.item())
        

In [6]:
fp = 1e-10
tp = 1e-10
fn = 1e-10
tn = 1e-10
with torch.no_grad():
    for images in val_loader:
        images = images.to(device)
        watermarks = torch.randint(0, 2, (images.shape[0], num_bits)).float().to(device)
        encoded_split = images.shape[0] // 2
        watermarks[:encoded_split] = 2
        images[encoded_split:] = encoder(watermarks[encoded_split:], images[encoded_split:])
        true_labels = (watermarks[:, 0] != 2).int()
        
        pred_scores = model(images)
        pred_labels = torch.argmax(pred_scores, dim=-1)
        
        tp += torch.sum((pred_labels == 1) & (pred_labels == true_labels)).item()
        tn += torch.sum((pred_labels == 0) & (pred_labels == true_labels)).item()
        fp += torch.sum((pred_labels == 1) & (pred_labels != true_labels)).item()
        fn += torch.sum((pred_labels == 0) & (pred_labels != true_labels)).item()
print(f1(tp, tn, fp, tn))

{'Precision': 0.9364406779657318, 'Recall': 0.48464912280702427, 'F1-score': 0.6387283236993417, 'Accuracy': 0.6458923512747049}
